In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE222073"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE222073"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE222073.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE222073.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE222073.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Patterns of metastasis and recurrence in urothelial cancer molecular subtypes"
!Series_summary	"This series contains the gene expression data from urothelial bladder cancer samples from Swedish patients that were used to analyze metastatic sites. Included patients have a recurrence or distant metastasis before or after treatment with chemotherapy. Patients with only lymph-node metastases are not included. A previous series (GSE169455) contains data from all patients that recieved two or more cycles of neoadjuvant chemotherapy with curative intent. Patients in that series that developed distant recurrence are re-analyzed here. A few samples from a previous cystectomy series (GSE83586) are also included as re-analysis. In addition, the current series contains data from patients treated with palliative first-line chemotherapy, curative adjuvant chemotherapy, or < 2 cycles of neoadjuvant chemotherapy."
!Series_summary	"Raw data should be adjusted in d

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, it appears this dataset contains gene expression data for urothelial bladder cancer
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Bladder Cancer)
# From the provided sample characteristics, we can use bone metastasis information as our trait
trait_row = 11  # Key 11 contains 'rm-bone: yes/no' data

# Age is not explicitly mentioned in the sample characteristics
age_row = None  

# Gender is not explicitly mentioned in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# For trait (bone metastasis in bladder cancer)
def convert_trait(value):
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 for yes, 0 for no)
    if value.lower() == 'yes':
        return 1
    elif value.lower() == 'no':
        return 0
    else:
        return None

# Age conversion function (not used as age is not available)
def convert_age(value):
    return None

# Gender conversion function (not used as gender is not available)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Check if the clinical data file exists
        if os.path.exists(f"{in_cohort_dir}/clinical_data.csv"):
            # Load the clinical data and extract features
            clinical_data = pd.read_csv(f"{in_cohort_dir}/clinical_data.csv")
            
            # Use the library function to extract features
            selected_clinical = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted features
            print(preview_df(selected_clinical))
            
            # Save the clinical data
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print(f"Clinical data file not found at {in_cohort_dir}/clinical_data.csv")
            print("Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        is_trait_available = False


Clinical data file not found at ../../input/GEO/Bladder_Cancer/GSE222073/clinical_data.csv
Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1-Mar', '2-Mar', '3-Mar', '4-Mar', '5-Mar', '6-Mar', '7-Mar', 'A2M',
       'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAT', 'AAGAB', 'AAK1',
       'AAMDC', 'AAMP', 'AANAT', 'AAR2'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the expression data

# Based on the sample of gene identifiers shown, I observe:
# - Many entries like "A2M", "AAAS", "AAMP" which appear to be standard HGNC gene symbols
# - Some unusual entries like "1-Mar", "2-Mar" etc. which are not standard gene symbols 
#   (these are likely MARCH family genes that have been incorrectly formatted)

# Since most identifiers appear to be gene symbols already but with some formatting issues,
# I'll recommend mapping to ensure consistency

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['LOC100287497', 'SAMD11', 'KLHL17', 'PLEKHN1', 'ISG15'], 'ORF': ['LOC100287497', 'SAMD11', 'KLHL17', 'PLEKHN1', 'ISG15']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. After observing the data, it seems that:
# - The gene expression data uses gene symbols directly as identifiers (e.g., A2M, AAAS)
# - The gene annotation data has 'ID' and 'ORF' columns which both contain gene identifiers

# Since the gene annotation preview data shows symbols like 'SAMD11', 'KLHL17', etc.
# which are standard gene symbols, I'll use 'ID' as both the probe column and the gene column
# for consistent mapping

# 2. Get a gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ORF')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the mapped gene data
print("Mapped gene data preview (first 5 genes):")
print(gene_data.index[:5])
print(f"Total number of genes after mapping: {len(gene_data)}")


Mapped gene data preview (first 5 genes):
Index(['A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')
Total number of genes after mapping: 13409


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data from the matrix file again to ensure we have the correct sample IDs
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

print("Clinical data structure:")
print(clinical_data.shape)
print("First few rows of clinical data:")
print(clinical_data.head())

# Extract clinical features with the correct sample IDs
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape after extraction: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Save the clinical data for inspection
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to match sample IDs between clinical and gene expression data."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = "Dataset contains gene expression data from bladder cancer samples with molecular subtyping information."
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=linked_data, 
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 13409
Normalized gene count: 13362


Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE222073.csv
Clinical data structure:
(13, 147)
First few rows of clinical data:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   
3  !Sample_characteristics_ch1   
4  !Sample_characteristics_ch1   

                                          GSM6914278  \
0  labeling kit: SensationPlus FFPE Amplification...   
1                                  labeling batch: 3   
2                      clinical tnm staging: cTxN0M1   
3                      chemotherapy type: palliative   
4                          lundtax rna-subtype: UroC   

                                          GSM6914279  \
0  labeling kit: SensationPlus FFPE Amplification...   
1                                  labeling batch: 4   
2                      clinical tnm staging: cT3N0M0   
3                     chemotherapy type: neoadjuvant   
4                

Linked data shape after handling missing values: (146, 13363)
For the feature 'Bladder_Cancer', the least common label is '1.0' with 53 occurrences. This represents 36.30% of the dataset.
The distribution of the feature 'Bladder_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bladder_Cancer/GSE222073.csv
